## Second Data Wrangling and Pre-processing
In the first notebook, we removed all of the data that could be revealing of an individual purchaser. <br>
In this notebook, we'll eliminate some unnecessary columns and create some more important feature columns that we can then look at in more detail in the Exploratory Data Analysis.

## Goal: Eliminate unnecessary columns, create some obvious features, minimize Nan values, and separate into Items, Orders, and Customers DataFrames

In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle

In [2]:
# change to the path with the raw csv file data

# load the pickled version of the 
df = pd.read_csv("cust_pub4_pydata.csv")
# look at the first 10 rows of this file
df.head(10)

/var/folders/_j/3_75d81x58lcmq4c6pqrpvp80000gn/T/ipykernel_90909/2830205445.py:4: DtypeWarning: Columns (21,35,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cust_pub4_pydata.csv")


,Unnamed: 0.1,Unnamed: 0,Name,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,...,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Receipt Number,Server,ship_bill,Area_Code,Cust_ID
0,58855,58855,#38539,paid,NaN,fulfilled,2023-04-27 18:06:51-07:00,no,USD,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,False,512.0,2.084681e+12
1,58856,58856,#38538,paid,2023-04-27 10:04:52-07:00,fulfilled,2023-04-30 18:24:14-07:00,no,USD,64.99,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,True,478.0,2.084670e+12
2,58857,58857,#38537,paid,2023-04-27 10:04:41-07:00,fulfilled,2023-04-30 18:24:34-07:00,yes,USD,168.34,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,True,NaN,2.084669e+12
3,58858,58858,#38537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,False,NaN,2.084669e+12
4,58859,58859,#38537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,False,NaN,2.084669e+12
5,58860,58860,#38537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,False,NaN,2.084669e+12
6,58861,58861,#38536,paid,2023-04-27 10:02:08-07:00,fulfilled,2023-04-30 18:24:34-07:00,no,USD,122.55,...,NaN,NaN,NaN,NaN,NaN,NaN,charter.net,True,NaN,2.084665e+12
7,58862,58862,#38536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,charter.net,False,NaN,2.084665e+12
8,58863,58863,#38535,paid,2023-04-27 10:00:05-07:00,fulfilled,2023-04-27 19:15:25-07:00,yes,USD,46.98,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,True,434.0,2.540498e+12
9,58864,58864,#38535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,gmail.com,False,NaN,2.540498e+12


In [3]:
# let's drop all of the tax columns from this DF
df.drop(['Tax 1 Name', 'Tax 1 Value', 'Tax 2 Name', 'Tax 2 Value', 'Tax 3 Name', 'Tax 3 Value', 'Tax 4 Name', 'Tax 4 Value',
       'Tax 5 Name', 'Tax 5 Value'], axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83369 entries, 0 to 83368
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0.1                 83369 non-null  int64  
 1   Unnamed: 0                   83369 non-null  int64  
 2   Name                         83369 non-null  object 
 3   Financial Status             37537 non-null  object 
 4   Paid at                      36195 non-null  object 
 5   Fulfillment Status           37537 non-null  object 
 6   Fulfilled at                 37295 non-null  object 
 7   Accepts Marketing            37537 non-null  object 
 8   Currency                     37537 non-null  object 
 9   Subtotal                     37537 non-null  float64
 10  Shipping                     37537 non-null  float64
 11  Taxes                        37537 non-null  float64
 12  Total                        37537 non-null  float64
 13  Discount Code   

In [5]:
# we noticed from the first 10 rows that some of these values aren't filled. Let's use forward fill since that is the same order
df['Paid at'].fillna(method='ffill', inplace=True, limit=None)

In [6]:
# we need to convert the "Paid at" column into datetime
df['Paid at'] = pd.to_datetime(df['Paid at'], infer_datetime_format=True)

In [7]:
df.head(10)

,Unnamed: 0.1,Unnamed: 0,Name,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,...,Id,Tags,Risk Level,Source,Lineitem discount,Receipt Number,Server,ship_bill,Area_Code,Cust_ID
0,58855,58855,#38539,paid,NaN,fulfilled,2023-04-27 18:06:51-07:00,no,USD,0.00,...,2.084681e+12,NaN,Low,shopify_draft_order,0.0,NaN,gmail.com,False,512.0,2.084681e+12
1,58856,58856,#38538,paid,2023-04-27 10:04:52-07:00,fulfilled,2023-04-30 18:24:14-07:00,no,USD,64.99,...,2.084670e+12,NaN,Low,web,0.0,NaN,gmail.com,True,478.0,2.084670e+12
2,58857,58857,#38537,paid,2023-04-27 10:04:41-07:00,fulfilled,2023-04-30 18:24:34-07:00,yes,USD,168.34,...,2.084669e+12,NaN,Low,web,0.0,NaN,gmail.com,True,NaN,2.084669e+12
3,58858,58858,#38537,NaN,2023-04-27 10:04:41-07:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,gmail.com,False,NaN,2.084669e+12
4,58859,58859,#38537,NaN,2023-04-27 10:04:41-07:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,gmail.com,False,NaN,2.084669e+12
5,58860,58860,#38537,NaN,2023-04-27 10:04:41-07:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,gmail.com,False,NaN,2.084669e+12
6,58861,58861,#38536,paid,2023-04-27 10:02:08-07:00,fulfilled,2023-04-30 18:24:34-07:00,no,USD,122.55,...,2.084665e+12,NaN,Low,web,0.0,NaN,charter.net,True,NaN,2.084665e+12
7,58862,58862,#38536,NaN,2023-04-27 10:02:08-07:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,charter.net,False,NaN,2.084665e+12
8,58863,58863,#38535,paid,2023-04-27 10:00:05-07:00,fulfilled,2023-04-27 19:15:25-07:00,yes,USD,46.98,...,2.084661e+12,NaN,Low,web,0.0,NaN,gmail.com,True,434.0,2.540498e+12
9,58864,58864,#38535,NaN,2023-04-27 10:00:05-07:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,gmail.com,False,NaN,2.540498e+12


In [8]:
# let's drop some more useless columns
df.drop(['Taxes', 'Notes', 'Note Attributes',
       'Cancelled at'], axis=1, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83369 entries, 0 to 83368
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0.1                 83369 non-null  int64  
 1   Unnamed: 0                   83369 non-null  int64  
 2   Name                         83369 non-null  object 
 3   Financial Status             37537 non-null  object 
 4   Paid at                      83368 non-null  object 
 5   Fulfillment Status           37537 non-null  object 
 6   Fulfilled at                 37295 non-null  object 
 7   Accepts Marketing            37537 non-null  object 
 8   Currency                     37537 non-null  object 
 9   Subtotal                     37537 non-null  float64
 10  Shipping                     37537 non-null  float64
 11  Total                        37537 non-null  float64
 12  Discount Code                5179 non-null   object 
 13  Discount Amount 

In [10]:
# Reciept Number is empty - drop that
# Fullfilled at is missing a lot of values - we are using 'Paid at '
# remove a few more columns that are too sparse to be useful in modeling
df.drop(['Fulfilled at', 'Receipt Number', 'Location', 'Device ID', 'Id', 'Risk Level'], axis=1, inplace=True)

In [11]:
# Let's see what currencies are used
df['Currency'].value_counts()

USD    37537
Name: Currency, dtype: int64

In [12]:
# it's just USD ($) or NaN. Not worth keeping that column
df.drop(['Currency'], axis=1, inplace=True)

In [13]:
# let's look at Paid at vs. Created at
df[['Paid at', 'Created at']].sample(10)

,Paid at,Created at
27882,2023-02-07 12:00:58-08:00,2023-02-07 12:00:58-08:00
79101,2020-06-08 22:52:29-07:00,2020-06-08 22:52:28-07:00
65315,2022-10-15 06:59:29-07:00,2022-10-15 06:59:28-07:00
8591,2023-04-10 16:52:33-07:00,2023-04-10 16:52:32-07:00
75055,2021-11-25 20:23:01-08:00,2021-11-25 20:23:00-08:00
28246,2023-02-06 08:45:07-08:00,2023-02-06 08:45:06-08:00
16802,2023-03-16 10:50:56-07:00,2023-03-16 10:50:54-07:00
59935,2022-11-14 07:11:44-08:00,2022-11-14 07:11:43-08:00
78215,2020-11-21 20:12:20-08:00,2020-11-21 20:12:20-08:00
18962,2023-03-06 14:43:56-08:00,2023-03-06 14:43:56-08:00


Those looks to be identical except for a 1-2 second lag for the payment. I'm good with dropping the paid at column

In [14]:
df.drop(['Paid at'], axis=1, inplace=True)

In [15]:
# since we are using 'Created at' as the time stamp, let's convert it to date time
# we need to convert the "Paid at" column into datetime
df['Created at'] = pd.to_datetime(df['Created at'], infer_datetime_format=True, utc=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83369 entries, 0 to 83368
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   Unnamed: 0.1                 83369 non-null  int64              
 1   Unnamed: 0                   83369 non-null  int64              
 2   Name                         83369 non-null  object             
 3   Financial Status             37537 non-null  object             
 4   Fulfillment Status           37537 non-null  object             
 5   Accepts Marketing            37537 non-null  object             
 6   Subtotal                     37537 non-null  float64            
 7   Shipping                     37537 non-null  float64            
 8   Total                        37537 non-null  float64            
 9   Discount Code                5179 non-null   object             
 10  Discount Amount              37537 non-null  f

### These look pretty good. Now, it's time to start filling in some of the NaN values

In [17]:
# For financial status
df['Financial Status'].value_counts()

paid                  36706
refunded                466
partially_refunded      356
partially_paid            5
pending                   4
Name: Financial Status, dtype: int64

In [18]:
# it looks like the first line of an order has that Financial Status; we'll forward fill
df['Financial Status'].fillna(method='ffill', inplace=True, limit=25)

In [19]:
# same applies for Fulfillment Status
df['Fulfillment Status'].fillna(method='ffill', inplace=True, limit=25)

In [20]:
# same is true for Accepts Marketing
df['Accepts Marketing'].fillna(method='ffill', inplace=True, limit=25)

In [21]:
df['Tags'].value_counts()

Subscription, Subscription First Order        511
recurring_order                               480
Subscription, Subscription Recurring Order    383
Local Pick-up                                   1
Name: Tags, dtype: int64

In [22]:
# these look unnecessarily complicated, so we'll drop - or maybe not
# df.drop(['Tags'], axis=1, inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83369 entries, 0 to 83368
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   Unnamed: 0.1                 83369 non-null  int64              
 1   Unnamed: 0                   83369 non-null  int64              
 2   Name                         83369 non-null  object             
 3   Financial Status             83362 non-null  object             
 4   Fulfillment Status           83362 non-null  object             
 5   Accepts Marketing            83362 non-null  object             
 6   Subtotal                     37537 non-null  float64            
 7   Shipping                     37537 non-null  float64            
 8   Total                        37537 non-null  float64            
 9   Discount Code                5179 non-null   object             
 10  Discount Amount              37537 non-null  f

In [24]:
# let's look at Payment Reference
df['Payment Reference'].value_counts()

c12320354861113.1    1
c11852344623161.1    1
c11852446466105.1    1
c11852434833465.1    1
c11852377161785.1    1
                    ..
c12076450840633.1    1
c12076480856121.1    1
#26189.1             1
c12076496912441.1    1
#1001.1              1
Name: Payment Reference, Length: 36557, dtype: int64

In [25]:
# let's drop it
df.drop(['Payment Reference'], axis=1, inplace=True)

In [26]:
# let's create one more feature that would be usable: total items in an order
df['ITEMS'] = df.groupby('Name')['Lineitem quantity'].transform('sum')

In [27]:
df['ITEMS'].unique()

array([  1,   4,   2,   3,   5,   6,   7,   8,  10,   9,  14,  80,  20,
        11,  40,  12,  19,  18,  30,  50,  52,  45,  15,  33,  16,  13,
       100,  25,  22,  35,  26,  21,  17])

In [28]:
# see how many unique "names" are in the DF
df['Name'].value_counts()

#5957     33
#34086    20
#25140    19
#10488    16
#28218    14
          ..
#16708     1
#16702     1
#16684     1
#16681     1
#1001      1
Name: Name, Length: 37537, dtype: int64

This looks like the same number of "subtotal" and some other fields that are order specific.

In [29]:
# let see if we can use the compare at price relative to the lineitem price as another feature
df['compared'] = (df['Lineitem compare at price'] - df['Lineitem price'])/df['Lineitem price']
# positive values mean the line item price is cheaper
# this relative price is more important than the absolute

In [30]:
# let's convert this to a difference in price
df['Lineitem compare at price'] = df['Lineitem compare at price'] - df['Lineitem price']

### Separate the Dataframe <br>
Right now, the items and the orders are each lines in the DataFrame; we are going to separate out the orders and items in the order into 2 separate dataframes:
### 1. Order - contains the order information
### 2. Items - line by line items contained in an order
### 3. Customers - contains the sum of the orders and items

In [31]:
# create Order DF by taking the first line of a name
Order = df.groupby('Name').first()

# or I could do it groupby Name and then take the value that has a subtotal that is not null

In [32]:
Order.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37537 entries, #10000 to #9999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   Unnamed: 0.1                 37537 non-null  int64              
 1   Unnamed: 0                   37537 non-null  int64              
 2   Financial Status             37537 non-null  object             
 3   Fulfillment Status           37537 non-null  object             
 4   Accepts Marketing            37537 non-null  object             
 5   Subtotal                     37537 non-null  float64            
 6   Shipping                     37537 non-null  float64            
 7   Total                        37537 non-null  float64            
 8   Discount Code                5179 non-null   object             
 9   Discount Amount              37537 non-null  float64            
 10  Shipping Method              36851 non-null  o

In [33]:
# let's look at discount codes
Order['Discount Code'].value_counts()

BAMBUBEAUTY           988
THANKYOU10            496
save10                351
Custom discount       224
CARACLARKNUTRITION    212
                     ... 
475f1cdff9cd            1
KJo108off               1
bad bottle              1
store credit            1
donation                1
Name: Discount Code, Length: 759, dtype: int64

The most popular discount codes are used largely enough that they could provide some value, but the largest code is used on 2% of all orders; discount codes are used on 11% of orders. I think it's best to just consider the discount amount to start and that's already contained in another column, so we'll drop this column.

In [34]:
Order.drop(['Discount Code'], axis=1, inplace=True)

In [35]:
# for orders, it shouldn't matter if that particular items is taxable, so we'll drop that or the fulfillment status
Order.drop(['Lineitem taxable', 'Lineitem fulfillment status'], axis=1, inplace=True)

In [36]:
Order.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37537 entries, #10000 to #9999
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   Unnamed: 0.1                37537 non-null  int64              
 1   Unnamed: 0                  37537 non-null  int64              
 2   Financial Status            37537 non-null  object             
 3   Fulfillment Status          37537 non-null  object             
 4   Accepts Marketing           37537 non-null  object             
 5   Subtotal                    37537 non-null  float64            
 6   Shipping                    37537 non-null  float64            
 7   Total                       37537 non-null  float64            
 8   Discount Amount             37537 non-null  float64            
 9   Shipping Method             36851 non-null  object             
 10  Created at                  37537 non-null  datetime64[ns,

In [37]:
# let's fill the payment method with "unknown for the missing values"
Order['Payment Method'].fillna(value='Unknown', inplace=True)

In [38]:
# let's look at Line item requires shipping
Order['Lineitem requires shipping'].value_counts()

True     22436
False    15101
Name: Lineitem requires shipping, dtype: int64

That seems reasonable enough; let's keep that

In [39]:
Order['Lineitem sku'].isna().sum()

2032

In [40]:
# let's drop some more unnecessary info; line item name should be covered in the sku
Order['Lineitem sku'].fillna(value=Order['Lineitem name'], inplace=True)
Order.drop(['Lineitem name'], axis=1, inplace=True)

In [41]:
Order.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37537 entries, #10000 to #9999
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   Unnamed: 0.1                37537 non-null  int64              
 1   Unnamed: 0                  37537 non-null  int64              
 2   Financial Status            37537 non-null  object             
 3   Fulfillment Status          37537 non-null  object             
 4   Accepts Marketing           37537 non-null  object             
 5   Subtotal                    37537 non-null  float64            
 6   Shipping                    37537 non-null  float64            
 7   Total                       37537 non-null  float64            
 8   Discount Amount             37537 non-null  float64            
 9   Shipping Method             36851 non-null  object             
 10  Created at                  37537 non-null  datetime64[ns,

In [42]:
# Customer ID should be an integer - but this gets weird, so we'll skip it.
# Order['Cust_ID'] = Order['Cust_ID'].astype('int')

In [43]:
# let's find out how this shipping method looks
Order['Shipping Method'].value_counts()

USPS First Class Package (5-7 Business Days)      14896
USPS First Class Package (2-5 Business Days)       7680
Standard Shipping (5-7 Business Days)              6486
Priority Mail                                      1299
First Class Package                                 925
Flat Rate Shipping                                  818
USPS Priority Mail (1-3 Business Days)              772
Standard Shipping (free)                            696
USPS First Class International                      503
Always Free Shipping                                480
Free shipping for orders over $99                   463
Free shipping                                       382
USPS First Class                                    216
2-3 Day Priority Mail (USPS)                        178
USPS First Class International Package              162
UPS® Ground                                         135
USPS                                                102
USPS Priority Mail                              

In [44]:
# let's fill that shipping method with unknown - Shipping Method
Order['Shipping Method'].fillna(value='Unknown', inplace=True)

In [45]:
Order.head(10)

,Unnamed: 0.1,Unnamed: 0,Financial Status,Fulfillment Status,Accepts Marketing,Subtotal,Shipping,Total,Discount Amount,Shipping Method,...,Employee,Tags,Source,Lineitem discount,Server,ship_bill,Area_Code,Cust_ID,ITEMS,compared
Name,,,,,,,,,,,,,,,,,,,,,
#10000,122994,122994,paid,fulfilled,yes,8.0,0.00,8.62,0.0,USPS First Class Package (2-5 Business Days),...,False,None,web,0.0,gmail.com,True,NaN,2.029549e+12,1,NaN
#10001,122992,122992,paid,fulfilled,yes,44.0,0.00,44.00,0.0,USPS First Class Package (2-5 Business Days),...,False,None,web,0.0,gmail.com,True,NaN,2.604838e+12,2,NaN
#10002,122991,122991,paid,fulfilled,yes,34.0,0.00,34.00,0.0,USPS First Class Package (2-5 Business Days),...,False,None,web,0.0,yahoo.com,False,813.0,1.928534e+12,1,-1.0
#10003,122990,122990,paid,fulfilled,yes,34.0,0.00,34.00,0.0,USPS First Class Package (2-5 Business Days),...,False,None,web,0.0,gmail.com,True,NaN,1.825239e+12,1,-1.0
#10004,122989,122989,paid,fulfilled,yes,8.0,0.00,8.00,0.0,USPS First Class Package (2-5 Business Days),...,False,None,web,0.0,gmail.com,True,513.0,1.886785e+12,1,NaN
#10005,122988,122988,paid,fulfilled,yes,34.0,14.55,48.55,0.0,USPS First Class International,...,False,None,web,0.0,gmail.com,True,NaN,1.825258e+12,1,-1.0
#10006,122986,122986,partially_refunded,fulfilled,yes,56.0,8.99,64.99,0.0,Priority Mail,...,False,None,web,0.0,gmail.com,False,NaN,1.825330e+12,2,NaN
#10007,122981,122981,paid,fulfilled,yes,80.0,0.00,80.00,0.0,USPS First Class Package (2-5 Business Days),...,False,None,web,0.0,gmail.com,False,NaN,1.825345e+12,5,NaN
#10008,122977,122977,paid,fulfilled,yes,72.0,0.00,72.00,8.0,USPS First Class Package (2-5 Business Days),...,False,None,web,0.0,gmail.com,True,NaN,2.010503e+12,4,NaN


In [46]:
# based on some weird data, let's look at the source
Order.Source.value_counts()

web                    35084
shopify_draft_order     1035
294517                   874
457101                   478
580111                    45
charge_rabbit             12
1424624                    4
iphone                     3
412739                     2
Name: Source, dtype: int64

shopify_draft_order may just be draft orders that were used to test the system and not actual orders

In [47]:
Order[Order['Source'] == 'shopify_draft_order']

,Unnamed: 0.1,Unnamed: 0,Financial Status,Fulfillment Status,Accepts Marketing,Subtotal,Shipping,Total,Discount Amount,Shipping Method,...,Employee,Tags,Source,Lineitem discount,Server,ship_bill,Area_Code,Cust_ID,ITEMS,compared
Name,,,,,,,,,,,,,,,,,,,,,
#10009,122976,122976,paid,fulfilled,yes,0.0,0.0,0.0,18.0,Free shipping,...,True,None,shopify_draft_order,0.0,gmail.com,True,347.0,2.110373e+12,1,NaN
#10010,122975,122975,paid,unfulfilled,yes,0.0,0.0,0.0,28.0,Free shipping,...,True,None,shopify_draft_order,0.0,yahoo.com,True,NaN,1.825382e+12,1,NaN
#10299,122473,122473,paid,fulfilled,yes,0.0,0.0,0.0,1192.0,Free shipping,...,True,None,shopify_draft_order,0.0,hold,True,949.0,2.615669e+12,15,NaN
#10308,122459,122459,paid,fulfilled,yes,0.0,0.0,0.0,46.0,Free shipping,...,True,None,shopify_draft_order,0.0,gmail.com,True,NaN,1.834192e+12,1,NaN
#10409,122305,122305,paid,fulfilled,yes,0.0,0.0,0.0,38.0,Free shipping,...,True,None,shopify_draft_order,0.0,gmail.com,True,NaN,1.982813e+12,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#9754,123438,123438,paid,fulfilled,yes,0.0,0.0,0.0,34.0,Free shipping,...,True,None,shopify_draft_order,0.0,gmail.com,False,570.0,1.836541e+12,1,NaN
#9755,123437,123437,paid,fulfilled,yes,0.0,0.0,0.0,28.0,Free shipping,...,True,None,shopify_draft_order,0.0,gmail.com,True,618.0,1.814752e+12,1,NaN
#9761,123430,123430,paid,fulfilled,yes,0.0,0.0,0.0,28.0,Free shipping,...,True,None,shopify_draft_order,0.0,yahoo.com,True,NaN,2.588982e+12,1,NaN


These look weird and are probably just tests. I'm dropping them.

In [48]:
Order = Order[~(Order['Source'] == 'shopify_draft_order')]

In [49]:
Order.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36502 entries, #10000 to #9999
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   Unnamed: 0.1                36502 non-null  int64              
 1   Unnamed: 0                  36502 non-null  int64              
 2   Financial Status            36502 non-null  object             
 3   Fulfillment Status          36502 non-null  object             
 4   Accepts Marketing           36502 non-null  object             
 5   Subtotal                    36502 non-null  float64            
 6   Shipping                    36502 non-null  float64            
 7   Total                       36502 non-null  float64            
 8   Discount Amount             36502 non-null  float64            
 9   Shipping Method             36502 non-null  object             
 10  Created at                  36502 non-null  datetime64[ns,

In [50]:
# accepts marketing is currently "yes" or "no"; it's much better if we consider them as 1 and 0 respectively
# then when we sum them up for multiple customer orders, it represents what went on better
Order['Accepts Marketing'].replace(to_replace='yes', value=1, inplace=True)
Order['Accepts Marketing'].replace(to_replace='no', value=0, inplace=True)

### I think that wraps it up for the Order DF

### On to the Items DF that contains all of the line items in the orders

In [51]:
# every row in the dataframe represents a line item, so we'll keep them in 
Items = df.copy()

### That takes care of the Items DF

### Still have to work on the Customer DF

In [52]:
Order['Cust_ID'].value_counts()

2.746662e+12    43
2.695320e+12    40
2.599750e+12    35
2.577523e+12    34
2.602749e+12    32
                ..
1.933791e+12     1
1.933785e+12     1
1.933783e+12     1
1.933769e+12     1
1.824887e+12     1
Name: Cust_ID, Length: 24135, dtype: int64

Let's separate the customers based on these value counts

In [53]:
#Order[Order['Cust_ID'] == -2147483648]
# this order showed up in 60k orders when we changed these from float to integer. I have non idea why

In [54]:
# Create customer DF by aggregating the orders DF over the Customer ID
# 'Accepts Marketing': 'mode', 'Shipping Method': 'mode', 'Payment Method': 'mode',
Cust = Order.groupby('Cust_ID', as_index=False).agg({'Total': ["sum", 'mean', 'first'], 'Fulfillment Status': 'count', 'Subtotal': 'sum', 'Shipping': 'sum', 'Refunded Amount': 'sum', 'Accepts Marketing': ['sum', 'first'], 'ITEMS': ['sum', 'mean', 'first'], 'Created at': ['first', 'last'], 'Server': 'first', 'Discount Amount': 'sum', 'Vendor': 'first', 'Employee': 'first', 'Source': 'first', 'ship_bill': 'first', 'Area_Code': 'first', 'Shipping Zip': 'first', 'Lineitem sku': 'first'})

In [55]:
Cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24135 entries, 0 to 24134
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   (Cust_ID, )                  24135 non-null  float64            
 1   (Total, sum)                 24135 non-null  float64            
 2   (Total, mean)                24135 non-null  float64            
 3   (Total, first)               24135 non-null  float64            
 4   (Fulfillment Status, count)  24135 non-null  int64              
 5   (Subtotal, sum)              24135 non-null  float64            
 6   (Shipping, sum)              24135 non-null  float64            
 7   (Refunded Amount, sum)       24135 non-null  float64            
 8   (Accepts Marketing, sum)     24135 non-null  int64              
 9   (Accepts Marketing, first)   24135 non-null  int64              
 10  (ITEMS, sum)                 24135 non-null  i

In [56]:
# this is exciting let's look at the first 10 rows
Cust.head(10)

Cust_ID   Total                 Fulfillment Status Subtotal Shipping  \
                sum    mean   first              count      sum      sum   
0   75306.0    0.00    0.00    0.00                  1     0.00      0.0   
1  112053.0  145.58  145.58  145.58                  1   145.58      0.0   
2  112055.0  137.55  137.55  137.55                  1   137.55      0.0   
3  112095.0   22.98   22.98   22.98                  1    22.98      0.0   
4  130108.0   28.00   28.00   28.00                  1    28.00      0.0   
5  130110.0   12.00   12.00   12.00                  1    12.00      0.0   
6  130188.0   42.00   42.00   42.00                  1    42.00      0.0   
7  130231.0   27.20   27.20   27.20                  1    27.20      0.0   
8  130241.0   22.00   22.00   22.00                  1    22.00      0.0   
9  130245.0  100.00  100.00  100.00                  1   100.00      0.0   

  Refunded Amount Accepts Marketing        ...                Created at  \
              sum               sum first  ...                      last   
0             0.0                 0     0  ... 2023-03-18 04:04:53+00:00   
1             0.0                 1     1  ... 2022-11-26 21:44:16+00:00   
2             0.0                 0     0  ... 2022-11-26 20:52:08+00:00   
3             0.0                 1     1  ... 2022-11-26 18:12:04+00:00   
4             0.0                 0     0  ... 2022-08-07 18:14:49+00:00   
5             0.0                 1     1  ... 2022-08-07 18:05:28+00:00   
6             0.0                 0     0  ... 2022-08-07 03:45:52+00:00   
7             0.0                 1     1  ... 2022-08-06 22:00:54+00:00   
8            22.0                 1     1  ... 2022-08-06 20:22:25+00:00   
9           100.0                 1     1  ... 2022-08-06 19:59:05+00:00   

   Server Discount Amount Vendor Employee  Source ship_bill Area_Code  \
    first             sum  first    first   first     first     first   
0  custom          154.00    0.0     True  iphone     False       NaN   
1  custom           16.17    1.0    False     web      True     404.0   
2  custom            0.00    1.0    False     web      True     845.0   
3  custom            0.00    1.0    False     web      True     262.0   
4  custom            0.00    0.0    False     web      True     617.0   
5  custom            0.00    0.0    False     web      True     740.0   
6  custom            0.00    0.0    False     web      True     701.0   
7  custom            6.80    0.0    False     web      True     754.0   
8  custom            0.00    0.0    False     web      True       NaN   
9  custom            0.00    0.0    False     web      True     617.0   

  Shipping Zip Lineitem sku  
         first        first  
0        32817      BES3001  
1        30087   ROUTEINS14  
2        12545   ROUTEINS13  
3        53402   ROUTEINS10  
4        01983      BEM1003  
5        43143      BEM2002  
6        58801      BES1006  
7        33026      BEM6001  
8        01880      BES5001  
9        01880      BEM1007  

[10 rows x 24 columns]

In [57]:
# multi-indexing can be a pain. I will reduce this to a single index
col = ['Cust_ID', 'Life_Total', 'Avg_Order', 'first_total', 'Orders', 'Sub_Total', 'Ship_Total', 'Refund_Total', 'Marketing_lf', 'Marketing_first', "Total_Items", 'Avg_Items', 'first_items', 'first_order', 'last_order', 'server', 'Disc_Total', 'Vendor', 'Emp', 'Source', 'ship_bill', 'Area_Code', 'Ship_Zip', 'lead_sku']
Cust.columns = col

In [58]:
# this is exciting let's look at the first 10 rows
Cust.head(10)

,Cust_ID,Life_Total,Avg_Order,first_total,Orders,Sub_Total,Ship_Total,Refund_Total,Marketing_lf,Marketing_first,...,last_order,server,Disc_Total,Vendor,Emp,Source,ship_bill,Area_Code,Ship_Zip,lead_sku
0,75306.0,0.00,0.00,0.00,1,0.00,0.0,0.0,0,0,...,2023-03-18 04:04:53+00:00,custom,154.00,0.0,True,iphone,False,NaN,32817,BES3001
1,112053.0,145.58,145.58,145.58,1,145.58,0.0,0.0,1,1,...,2022-11-26 21:44:16+00:00,custom,16.17,1.0,False,web,True,404.0,30087,ROUTEINS14
2,112055.0,137.55,137.55,137.55,1,137.55,0.0,0.0,0,0,...,2022-11-26 20:52:08+00:00,custom,0.00,1.0,False,web,True,845.0,12545,ROUTEINS13
3,112095.0,22.98,22.98,22.98,1,22.98,0.0,0.0,1,1,...,2022-11-26 18:12:04+00:00,custom,0.00,1.0,False,web,True,262.0,53402,ROUTEINS10
4,130108.0,28.00,28.00,28.00,1,28.00,0.0,0.0,0,0,...,2022-08-07 18:14:49+00:00,custom,0.00,0.0,False,web,True,617.0,01983,BEM1003
5,130110.0,12.00,12.00,12.00,1,12.00,0.0,0.0,1,1,...,2022-08-07 18:05:28+00:00,custom,0.00,0.0,False,web,True,740.0,43143,BEM2002
6,130188.0,42.00,42.00,42.00,1,42.00,0.0,0.0,0,0,...,2022-08-07 03:45:52+00:00,custom,0.00,0.0,False,web,True,701.0,58801,BES1006
7,130231.0,27.20,27.20,27.20,1,27.20,0.0,0.0,1,1,...,2022-08-06 22:00:54+00:00,custom,6.80,0.0,False,web,True,754.0,33026,BEM6001
8,130241.0,22.00,22.00,22.00,1,22.00,0.0,22.0,1,1,...,2022-08-06 20:22:25+00:00,custom,0.00,0.0,False,web,True,NaN,01880,BES5001
9,130245.0,100.00,100.00,100.00,1,100.00,0.0,100.0,1,1,...,2022-08-06 19:59:05+00:00,custom,0.00,0.0,False,web,True,617.0,01880,BEM1007


In [59]:
Cust['Orders'].value_counts()

1     18572
2      3115
3      1136
4       545
5       285
6       147
7        87
8        52
9        44
10       20
14       18
11       17
12       16
13       15
16       14
19        9
17        7
15        7
20        4
18        4
26        3
23        2
25        2
21        2
31        2
24        2
28        1
40        1
27        1
35        1
22        1
32        1
34        1
43        1
Name: Orders, dtype: int64

In [60]:
Order.groupby('Cust_ID')['Created at'].min()

Cust_ID
7.530600e+04   2023-03-18 04:04:53+00:00
1.120530e+05   2022-11-26 21:44:16+00:00
1.120550e+05   2022-11-26 20:52:08+00:00
1.120950e+05   2022-11-26 18:12:04+00:00
1.301080e+05   2022-08-07 18:14:49+00:00
                          ...           
2.777047e+12   2022-11-15 13:35:57+00:00
2.777047e+12   2022-11-18 16:57:49+00:00
2.777048e+12   2023-02-24 15:43:24+00:00
2.777050e+12   2022-11-20 12:21:13+00:00
2.779013e+12   2023-04-10 11:04:35+00:00
Name: Created at, Length: 24135, dtype: datetime64[ns, UTC]

In [61]:
Order['1st'] = Order['Created at'] == Order['Created at']

In [62]:
#Order['first_6mon'] = Order['']

In [63]:
 # Create customer DF by aggregating the orders DF over the Customer ID
# 'Accepts Marketing': 'mode', 'Shipping Method': 'mode', 'Payment Method': 'mode',
cust2 = Order.groupby('Cust_ID',  as_index=False).apply(lambda g: g.sort_values('Created at'))

I think that does it for data wrangling. Let's export the data so that we can do EDA in the next notebook.

## Set timezones

In [64]:
def set_timezone(df, date_cols):
    for date_col in date_cols:
        df[date_col] = pd.to_datetime(df[date_col], utc=True)
        df[date_col] = df[date_col].dt.tz_convert('US/Pacific')
        return df
    
Cust = set_timezone(Cust, ["first_order", "last_order"])
Items = set_timezone(Items, ["Created at"])
Order = set_timezone(Order, ["Created at"])

In [70]:
Order.to_csv("../order.csv")

In [66]:
Items.to_csv("../items.csv")

In [67]:
Cust.to_csv("../cust.csv")

See you in the EDA